# Scratch

Scratch notebook for testing code.

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
repo_path = '/Users/mincheolkim/Github/'
data_path = '/Users/mincheolkim/Documents/'

In [3]:
import sys
sys.path.append(repo_path + 'scVI')
sys.path.append(repo_path + 'scVI-extensions')

In [4]:
import torch
import imp

In [5]:
import os

import numpy as np
from sklearn.manifold import TSNE
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import seaborn as sns

from scvi.metrics.clustering import entropy_batch_mixing, get_latent
from scvi.models import VAE, SVAEC
from scvi.inference import VariationalInference

/anaconda3/envs/scvi/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
import pandas as pd

In [7]:
import scvi_extensions.dataset.cropseq as cs

In [8]:
imp.reload(cs)

<module 'scvi_extensions.dataset.cropseq' from '/Users/mincheolkim/Github/scVI-extensions/scvi_extensions/dataset/cropseq.py'>

In [11]:
# Load the model
model_path = data_path + 'vae_model_test.model'
vae = torch.load(model_path, map_location=lambda storage, loc: storage)

/anaconda3/envs/scvi/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/anaconda3/envs/scvi/lib/python3.6/site-packages/torch/serialization.py:425: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm1d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [9]:
# Load the dataset
h5_filename = data_path + 'raw_gene_bc_matrices_h5.h5'
metadata_filename = data_path + 'nsnp20.raw.sng.km_vb1.norm.meta.txt'
gene_dataset = cs.CropseqDataset(
    filename=h5_filename,
    metadata_filename=metadata_filename,
    use_donors=True,
    use_labels=True,
    save_path='')

Preprocessing CROP-seq dataset lalala
Number of cells kept after filtering with metadata: 283634
Number of cells kept after removing all zero cells: 283634
Finished preprocessing CROP-seq dataset


In [12]:
# Create an inference class
use_batches=True
use_cuda=True
infer = VariationalInference(
    vae, 
    gene_dataset, 
    train_size=0.9, 
    use_cuda=use_cuda,
    verbose=True,
    frequency=5)

In [18]:
gene_dataset.labels

array([[280],
       [ 40],
       [280],
       ...,
       [280],
       [ 40],
       [116]])

In [29]:
latent = []
for tensors in infer.data_loaders['sequential']:
    sample_batch, local_l_mean, local_l_var, batch_index, label = tensors
    latent += [vae.sample_from_posterior_z(sample_batch, y=label)]
    break

In [75]:
torch.distributions.Normal(torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])).rsample()

tensor([1.8844, 0.3331])

In [82]:
x = sample_batch

In [122]:
q = vae.z_encoder.encoder(x)[0, :]

In [130]:
var = vae.z_encoder.var_encoder(vae.z_encoder.encoder(x)[10, :])

In [137]:
torch.exp(torch.clamp(var, -5, 5))

tensor([0.0067, 0.0067, 0.0067, 0.0067, 0.0067, 0.0067, 0.0067, 0.0067, 0.0067,
        0.0067])

In [120]:
q

tensor([[ 2.5654,  0.0000,  0.0000,  ...,  2.5109,  8.8773,  0.7601],
        [ 0.0000,  0.0000,  0.0000,  ..., 15.8079,  0.0000,  0.0000],
        [ 7.5503,  0.0000,  0.0000,  ...,  1.1686, 13.2909,  5.0562],
        ...,
        [ 8.6540, 18.0296,  0.0000,  ..., 11.6259,  9.6111,  2.0203],
        [ 0.0000,  0.0000,  0.0000,  ...,  9.4792,  1.8050,  0.0000],
        [ 2.3887,  0.0000,  0.0000,  ...,  6.2484,  9.2351,  0.0000]])

In [99]:
vae.z_encoder.var_encoder.weight.shape

torch.Size([10, 128])

In [93]:
qz_m

tensor([[ -81.9397,  -76.3242,  -16.1560,  ...,  124.1240,  271.2110,
          300.8079],
        [-163.5321,   72.3545, -374.5663,  ...,  271.3742,  156.3855,
          163.5798],
        [ -92.2339,  -57.9898,  -87.6369,  ...,  242.6978,  124.5783,
          185.8195],
        ...,
        [  19.6231,  -21.3858, -393.1049,  ...,  475.5630, -191.9860,
          172.7761],
        [-153.8749,  -48.3089, -190.6447,  ...,  183.3277,  372.9952,
          343.9473],
        [-163.2225, -111.5427,  -87.5208,  ...,  187.7696,  255.4589,
          297.7498]])

In [62]:
vae.sample_from_posterior_z(sample_batch, y=label)[0, :]

tensor([10.2543, -7.8392, -4.8887, -5.5117, -9.9770, 18.0329, 36.2084,  3.4973,
        16.5866, 28.1041])

In [44]:
sample_batch.shape

torch.Size([128, 32738])

In [35]:
latent[0].shape

torch.Size([128, 10])

In [40]:
latent[0].data.numpy()[3, :]

array([-0.84022033,  4.3386154 , -3.2906535 , -4.476451  , -2.4687972 ,
       24.499353  , 14.695851  , 30.992464  , 20.389019  ,  5.676299  ],
      dtype=float32)

In [24]:
temp = sample_batch.data.numpy()

In [28]:
temp.sum(axis=1)

array([ 1137.,  5472.,  3480.,  1732.,  4473.,  4439.,  2447.,  1142.,
        7492.,  5171.,  5632.,  2619.,   720.,  7774.,  1244.,  1294.,
        1930.,  4889.,  3251.,  1209.,  5922.,  2607.,  3778.,  4994.,
        5410.,  1198.,  3473.,  1982., 20290.,  7548.,  3356.,  4367.,
        1278.,  1978.,  8913.,  3520.,  1368.,  1884.,  1585.,  1699.,
        5984.,  1647.,  1657.,  2604.,  1476.,  1934., 10458.,  4358.,
        4806.,  2660.,  3067.,  2884.,  5543.,  1202.,  3372.,  2360.,
        1205.,  2088.,  1668.,  2356.,  1868.,  1515.,  2080.,  2059.,
        1903.,  1789.,  1975.,  2034.,  6618.,  9371.,  3633.,  1854.,
        1867.,  5155.,  2370.,  1837.,  1446.,  3071.,  1544.,  3738.,
        1993.,  2015.,  1248.,   995., 15023.,  7215.,  2474.,  5868.,
        1895.,  3151.,  2103.,  2477.,  2236., 11979.,  1250.,  2358.,
        2198.,  1611.,  3272.,  1904.,  2200.,  5870.,  3375.,  3827.,
        2522.,  2293.,  4947.,  2171.,  1769.,  8345.,  2050.,  1225.,
      

In [ ]:
latent 